# MyClimateFuture
This script is used to convert the CSV files to the JSON file that we use on the website.

In [6]:
using CSV
using DataFrames
using JSON

First, some basic settings about the data:

In [18]:
MAX_AGE = 60
AGES = [0:1:MAX_AGE;]

TEMPERATURE_VALUES = [0.87, 0.95, 1.04, 1.12, 1.21, 1.29, 1.38, 1.5, 1.61, 1.73, 1.82, 1.91, 2, 2.09, 2.18, 2.27, 2.36, 2.46, 2.57, 2.67, 2.77, 2.88, 2.98, 3.08, 3.19, 3.29, 3.4, 3.5]

RELEVANT_TEMPERATURES = [1.5, 2.36, 3.5]

3-element Array{Float64,1}:
 1.5 
 2.36
 3.5 

In [5]:
REGIONS = ("World", "East Asia & Pacific", "Europe & Central Asia", "Latin America & Caribbean", "Middle East & North Africa", "North America", "South Asia", "Sub-Saharan Africa")

("World", "East Asia & Pacific", "Europe & Central Asia", "Latin America & Caribbean", "Middle East & North Africa", "North America", "South Asia", "Sub-Saharan Africa")

In [8]:
RISKS = ("burntarea", "floodedarea", "cropfailedarea", "tropicalcyclonedarea", "driedarea", "heatwavedarea")

("burntarea", "floodedarea", "cropfailedarea", "tropicalcyclonedarea", "driedarea", "heatwavedarea")

This function gets the index for a specific temperature. Because we are only interested in 3 temperature values on the website, we convert them into indices to locate the values in the long data rows.

In [19]:
function getIndicesFromValues(temperature)
    findfirst(isequal(temperature), TEMPERATURE_VALUES)
end

TEMPERATURE_INDICES = map(x -> getIndicesFromValues(x), RELEVANT_TEMPERATURES)

3-element Array{Int64,1}:
  8
 17
 28

This function extracts the value from the specific temperature and age.

In [22]:
function getValueFromTempAndAge(temperature, age, data)
    parse(Float64, string(data[size(TEMPERATURE_VALUES, 1) - (temperature - 1), (MAX_AGE + 1) - age + 1]))
end

getValueFromTempAndAge (generic function with 1 method)

Next, let’s load a sample data set and use the function to recieve some data.

In [23]:
sample = CSV.read("source/02_EMF_young2pic_burntarea_World.csv", DataFrame, decimal=',')

getValueFromTempAndAge(getIndicesFromValues(1.5), 60, sample)

1.2

The next cell runs over all regions, risks, temperature indices and ages and stores the data in a nested arrays. It then writes these arrays to a JSON file.

In [24]:
rs = Array[]
for r in REGIONS
    is = Array[]
    for i in RISKS
        # println("Getting data for $(r) of type $(i)")
        datum = CSV.read("source/02_EMF_young2pic_$(i)_$(r).csv", DataFrame, decimal=',')
        ts = Array[]
        for t in TEMPERATURE_INDICES
            as = Float64[]
            for a in AGES
                value = getValueFromTempAndAge(t, a, datum)
                # println("$(r) $(i) $(t) $(a): $(value)")
                push!(as, value)
            end
            push!(ts, as)
        end
        push!(is, ts)
    end
    push!(rs, is)
end

open(string("data.json"), "w") do io
    write(io, JSON.json(rs));
end;

The next cell is used to find out the order of the bars. We want the bars to get higher on average. So we summarise the values for each risk and then sort these values.

In [25]:
sums = Tuple{String, Float32}[]
for risk in RISKS
    sum = 0
    for region in REGIONS
        datum = CSV.read("source/02_EMF_young2pic_$(risk)_$(region).csv", DataFrame, decimal=',')
        for temperature in TEMPERATURE_INDICES
            for age in AGES
                value = getValueFromTempAndAge(temperature, age, datum)
                sum += value
            end
        end
    end
    push!(sums, (risk, sum))
end

sort!(sums, by = x -> x[2]);
sums

6-element Array{Tuple{String,Float32},1}:
 ("burntarea", 2188.5)           
 ("floodedarea", 3188.5)         
 ("cropfailedarea", 4323.3)      
 ("tropicalcyclonedarea", 4649.8)
 ("driedarea", 7291.4)           
 ("heatwavedarea", 29102.6)      